In [ ]:
import plotly.graph_objects as go
import mdtraj as md

#----------------------------------------------------------------------------------------------------------------------
# this script can visualize the 3d coordinates of the residues in a pdb file and draw lines between the contacts between them
# the plot is interactive and residues and lines can be toggled on and off in order to see the contact matrix in a configuration
# you can load the contacts from a standard mdtraj residue selection its currently set to run with a smog-server .contacts file 
# to use mdtraj you can comment out lines 22 and 23 then uncomment lines 26 - 30 and make your contact selections, its worth noting that 
# using an mdtraj residue selection won't offer as much insight into the native structure of your protein as using the contacts file
# which can be easily generated at https://smog-server.org/cgi-bin/GenTopGro.pl
#----------------------------------------------------------------------------------------------------------------------
# Load PDB file
traj = md.load('/mnt/c/users/t_m_w/downloads/1ubq145.xtc', top='/mnt/c/users/t_m_w/moleculardynamics/1ubqcalpha/1ubqcaonly.pdb')

# Get atom coordinates
atoms = traj.xyz[0]

# Create 3D plot
fig = go.Figure(data=[go.Scatter3d(x=atoms[:, 0], y=atoms[:, 1], z=atoms[:, 2], mode='markers', opacity = 0.5)])

# Get residue pairs from contacts file
contacts_df = pd.read_csv('/mnt/c/users/t_m_w/moleculardynamics/1ubqcalpha/1ubqca.contacts', sep='\s+', usecols=[1, 3], header=None)
residue_pairs = contacts_df.values.tolist()

# Get residue pairs from mdtraj 
#heavy = native.topology.select_atom_indices('heavy')
#heavy_pairs = np.array(
#        [(i,j) for (i,j) in combinations(heavy, 2)
#            if abs(native.topology.atom(i).residue.index - \
#                   native.topology.atom(j).residue.index) > 3])

colors =  ['red', 'blue', 'green', 'yellow', 'purple', 'orange']
# Plot residues and lines between them

for i, pair in enumerate(residue_pairs):
    res1 = traj.topology.select(f'resid {pair[0]}')
    res2 = traj.topology.select(f'resid {pair[1]}')
    res1_atoms = traj.atom_slice(res1, inplace=False)
    res2_atoms = traj.atom_slice(res2, inplace=False)
    fig.add_trace(go.Scatter3d(x=res1_atoms.xyz[0][:, 0], y=res1_atoms.xyz[0][:, 1], z=res1_atoms.xyz[0][:, 2], mode='markers', marker=dict(color=colors[i % len(colors)])))
    fig.add_trace(go.Scatter3d(x=res2_atoms.xyz[0][:, 0], y=res2_atoms.xyz[0][:, 1], z=res2_atoms.xyz[0][:, 2], mode='markers', marker=dict(color=colors[i % len(colors)])))
    fig.add_trace(go.Scatter3d(x=[res1_atoms.xyz[0][0, 0], res2_atoms.xyz[0][0, 0]], y=[res1_atoms.xyz[0][0, 1], res2_atoms.xyz[0][0, 1]], z=[res1_atoms.xyz[0][0, 2], res2_atoms.xyz[0][0, 2]], mode='lines', line=dict(color=colors[i % len(colors)])))
    fig.add_annotation(x=(res1_atoms.xyz[0][0, 0] + res2_atoms.xyz[0][0, 0]) / 2, y=(res1_atoms.xyz[0][0, 1] + res2_atoms.xyz[0][0, 1]) / 2, text=f"{traj.topology.residue(res1[0]).name} - {traj.topology.residue(res2[0]).name}", showarrow=False)

# Show plot
fig.show()